# Fusion Cyber AI Takehome Notebook
Ibrahim Musaddequr Rahman

References: https://github.com/koji/llm_api_template/blob/main/Zephyr_API_server.ipynb
https://python.langchain.com/docs/integrations/llms/llamacpp

In [17]:
!pip install langchain

In [18]:
# can be replaced with other models
!wget https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q2_K.gguf

--2024-04-04 01:12:05--  https://huggingface.co/TheBloke/Llama-2-7B-GGUF/resolve/main/llama-2-7b.Q2_K.gguf
Resolving huggingface.co (huggingface.co)... 99.84.108.55, 99.84.108.87, 99.84.108.129, ...
Connecting to huggingface.co (huggingface.co)|99.84.108.55|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/90/07/90076ae9a201487aedadb49bde2070797e223829cae7492b17e60c2fd791b379/a630f354771cf25496e079a49656730858712315cc71aee4adf9b97aceb251f8?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27llama-2-7b.Q2_K.gguf%3B+filename%3D%22llama-2-7b.Q2_K.gguf%22%3B&Expires=1712452326&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxMjQ1MjMyNn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy85MC8wNy85MDA3NmFlOWEyMDE0ODdhZWRhZGI0OWJkZTIwNzA3OTdlMjIzODI5Y2FlNzQ5MmIxN2U2MGMyZmQ3OTFiMzc5L2E2MzBmMzU0NzcxY2YyNTQ5NmUwNzlhNDk2NTY3MzA4NTg3MTIzMTVjYzcxYWVlNGFkZjliOTdhY

In [19]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

llm = LlamaCpp(
    model_path="llama-2-7b.Q2_K.gguf",
    temperature=0.75,
    # small amount of tokens so it runs fast, having major performance issues
    max_tokens=10,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,  # Verbose is required to pass to the callback manager
)


llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from llama-2-7b.Q2_K.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = LLaMA v2
llama_model_loader: - kv   2:                       llama.context_length u32              = 4096
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 11008
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32              = 32
llam

Test Base Model

In [20]:
question = """
Question: A rap battle between Stephen Colbert and John Oliver
"""
llm.invoke(question)


I saw a video of a rap battle between


llama_print_timings:        load time =    7782.65 ms
llama_print_timings:      sample time =       5.94 ms /    10 runs   (    0.59 ms per token,  1684.92 tokens per second)
llama_print_timings: prompt eval time =   13467.93 ms /    16 tokens (  841.75 ms per token,     1.19 tokens per second)
llama_print_timings:        eval time =    5387.27 ms /     9 runs   (  598.59 ms per token,     1.67 tokens per second)
llama_print_timings:       total time =   18927.92 ms /    25 tokens


'\nI saw a video of a rap battle between'

Test Langchain

In [21]:
template = """Question: {question}

Answer: Let's work this out in a step by step way to be sure we have the right answer."""

prompt = PromptTemplate.from_template(template)

In [22]:
chain = prompt | llm

In [23]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"
chain.invoke({"question":question})


Llama.generate: prefix-match hit



2015 → 201


llama_print_timings:        load time =    7782.65 ms
llama_print_timings:      sample time =       6.30 ms /    10 runs   (    0.63 ms per token,  1586.55 tokens per second)
llama_print_timings: prompt eval time =   20751.95 ms /    44 tokens (  471.64 ms per token,     2.12 tokens per second)
llama_print_timings:        eval time =    5265.18 ms /     9 runs   (  585.02 ms per token,     1.71 tokens per second)
llama_print_timings:       total time =   26106.52 ms /    53 tokens


'\n2015 → 201'

Install Flask and Ngrok

In [24]:
# !wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# !unzip -o ngrok-stable-linux-amd64.zip
!curl -s https://ngrok-agent.s3.amazonaws.com/ngrok.asc | sudo tee /etc/apt/trusted.gpg.d/ngrok.asc >/dev/null && echo "deb https://ngrok-agent.s3.amazonaws.com buster main" | sudo tee /etc/apt/sources.list.d/ngrok.list && sudo apt update && sudo apt install ngrok
!pip install pyngrok

deb https://ngrok-agent.s3.amazonaws.com buster main
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ngrok-agent.s3.amazonaws.com buster InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 229 kB in 2s (145 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
45 p

In [25]:
!pip install flask

Add Ngrok Token, from Google Colab Secret

In [26]:
from google.colab import userdata
from IPython import get_ipython
get_ipython().system_raw(f'ngrok config add-authtoken {userdata.get("NGROK")}')

Create a basic API to send prompts to the LLM

In [27]:
# to integrate with langchain, I'm going to write a basic flask endpoint that does a similar thing to llama_cpp.server
from flask import Flask, request
import json
from threading import Thread
import os
from pyngrok import ngrok, conf


app = Flask(__name__)

@app.route('/prompt', methods=['GET','POST'])
def run_llama():
    data = request.get_json()
    prompt = data.get("prompt", "")
    # tokens = data.get("max_tokens", 50)

    response = chain.invoke(prompt)

    # Prepare and send the response back as JSON
    return json.dumps({"response": response})
@app.route("/")
def hello():
  return "<h1>hello<h1>"

def run_api():
    app.run()

def start_ngrok():
    # Open a ngrok tunnel to the Flask server
    url = ngrok.connect(5000)
    print(' * Tunnel URL:', url)

# if __name__ == "__main__":
    # Run the API in a separate thread to avoid blocking the notebook
api_thread = Thread(target=run_api)
api_thread.start()

start_ngrok()

 * Serving Flask app '__main__'
 * Debug mode: off
 * Tunnel URL: NgrokTunnel: "https://fd96-35-236-248-237.ngrok-free.app" -> "http://localhost:5000"


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


In [28]:
!curl -s http://localhost:4040/api/tunnels | python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

https://5393-35-236-248-237.ngrok-free.app


Kill Ngrok(for Debugging)

In [29]:
# !pkill uvicorn
# !pkill ngrok